In [1]:
import re

In [236]:
# test out regexs for NEB enzyme patterns
# reg_pattern = r'(?P<front>\([\-\d/]*\))*(?P<split1>[ACGTURYKMSWBDHVN]*)(?P<divided>[/])*(?P<split2>[ACGTURYKMSWBDHVN]*)(?P<back>\([\-\d/]*\))*\s+(?P<name>[\d\w]+)'
reg_pattern = r'(?P<front>\([\-\d/]*\))*(?P<split1>[ACGTURYKMSWBDHVN]*)(?P<divided>[/])*(?P<split2>[ACGTURYKMSWBDHVN]*)(?P<back>\([\-\d/]*\))*'
print(reg_pattern)
# m = re.match(reg_pattern, '(10/12)CGANNNNNNTGC(12/10)   BcgI')
m = re.match(reg_pattern, 'GACGT/C')
m.groupdict()

(?P<front>\([\-\d/]*\))*(?P<split1>[ACGTURYKMSWBDHVN]*)(?P<divided>[/])*(?P<split2>[ACGTURYKMSWBDHVN]*)(?P<back>\([\-\d/]*\))*


{'front': None, 'split1': 'GACGT', 'divided': '/', 'split2': 'C', 'back': None}

In [161]:
enzymes = {}

for line in open("restriction_table_full_clean.csv"):
    sp = line.split(",")
    m = re.match(reg_pattern, sp[1])
    enzymes[sp[0]] = [m.groupdict(),sp[0],sp[3],sp[4],sp[5],sp[6],sp[7],sp[8]]

In [164]:
enzymes['AgeI-HF']

[{'front': None,
  'split1': 'A',
  'divided': '/',
  'split2': 'CCGGT',
  'back': None},
 'AgeI-HF',
 '100',
 '50',
 '10',
 '100',
 '65C',
 '37C']

In [230]:
import math

fasta_mapping = {'A': 'A','C': 'C','G': 'G','T': 'T','U': 'T','R': '[AG]','Y': '[CT]','K': '[GT]','M': '[AC]','S': '[CG]',
                'W': '[AT]','B': '[CGT]','D': '[AGT]','H': '[ACT]','V': '[ACG]','N': '[ACGT]'}

def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False
    
class REnzyme:
    def __init__(self,mapping,name,buf11,buf21,buf31,cutsmart,inactivate,incubate):
        self.name = name.replace('-','_').replace('.','_')
        self.buffers = [buf11,buf21,buf31,cutsmart]
        self.temps = [incubate,inactivate]
        if mapping['divided'] == None:
            self.sequence = mapping['split1']
            self.cutsite_offset = 0
        else:
            self.sequence = mapping['split1'] + mapping['split2']
            self.cutsite_offset = len(mapping['split1'])
            
        self.upstream_tag = mapping['front']
        self.downstream_tag = mapping['back']
        self.regex_seq = re.compile(REnzyme.create_re(self.name,self.sequence))
        self.istypetwo = mapping['front'] != None or mapping['back'] != None
        #print(self.regex_seq)
    def create_re(name,sequence):
        return('(?P<' + name + '>'+ "".join([fasta_mapping[x] for x in sequence]) + ')')
    def overhang_length(self):
        return float(self.cutsite_offset) - (float(len(self.sequence))/2.0)
    def ideal(self):
        real_buffers = [int(x) if RepresentsInt(x) else 0 for x in self.buffers]
        working_temp_ok = self.temps[0] == '37C'
        inactive = self.temps[1] != 'No'
        return(real_buffers[3] > 90 and working_temp_ok and inactive)
    def type_two_s(self):
        return(self.istypetwo)
enz = REnzyme(enzymes['AgeI-HF'][0],'AgeI-HF','100','50','10','100','65C','37C')
m = re.findall(enz.regex_seq, 'GGAACCGGATCCGGAACC')
m
enz.overhang_length()
enz.ideal()

True

In [231]:
all_enzymes = {}
for enz,mping in enzymes.items():
    all_enzymes[enz] = REnzyme(mping[0],mping[1],mping[2],mping[3],mping[4],mping[5],mping[6],mping[7])

In [174]:
# for each sequence in the input, for each restriction enzyme, figure out how many matches we have

header = training_data.readline()
total_enzyme_counts = {}
total_enzyme_counts_per = {}
enzymes_to_sequences = {}
all_sequences = []
training_data = open("train_values.csv")
for index,line in enumerate(training_data):
    sp = line.split(",")
    if index % 10000 == 0:
        print(index)
    sequence = sp[1]
    sequence_name = sp[0]
    all_sequences.append(sequence_name)
    if len(sequence) > 2000:
        for enz_name, enz in all_enzymes.items():
            count = len(re.findall(enz.regex_seq, line))
            total_enzyme_counts[enz_name] = total_enzyme_counts.get(enz_name,0) + count
            total_enzyme_counts_per.setdefault(enz_name,list()).append(count)
            if count == 1 or count == 2:
                enzymes_to_sequences.setdefault(enz_name,list()).append(sequence_name)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000


In [199]:
len(all_enzymes)

284

In [177]:
import statistics 

In [232]:
enzymes_to_use = {}
min_overhang = 1
max_overhang = 2

for enz,count in total_enzyme_counts.items():
    med = statistics.median(total_enzyme_counts_per[enz])
    if enz in enzymes_to_sequences:
        min_over_check = (all_enzymes[enz].overhang_length() <= (-1 * min_overhang) or all_enzymes[enz].overhang_length() >= min_overhang)
        max_over_check = (all_enzymes[enz].overhang_length() >= (-1 * max_overhang) or all_enzymes[enz].overhang_length() <= max_overhang)
        if med <= 1 and max_over_check and min_over_check and all_enzymes[enz].ideal() and not all_enzymes[enz].type_two_s():
            print(enz + " " + str(count/63018) + " median " + str(med) + " set size " + str(len(enzymes_to_sequences[enz])))
            enzymes_to_use[enz] = set(enzymes_to_sequences[enz])
        
print(len(enzymes_to_use))

AflII 0.6756482274905583 median 1 set size 16167
AgeI-HF 0.4435716779332889 median 0 set size 16404
AscI 0.17195087117966296 median 0 set size 7782
AsiSI 0.1263607223333016 median 0 set size 7066
BclI-HF 0.905106477514361 median 1 set size 17983
BmtI-HF 0.5033482497064331 median 1 set size 20092
BspDI 0.593243200355454 median 1 set size 18559
BsrGI-HF 0.8606905963375544 median 1 set size 18334
Bsu36I 0.5141070805166778 median 0 set size 13399
ClaI 0.593243200355454 median 1 set size 18559
EcoRI-HF 0.845710749309721 median 1 set size 24022
FseI 0.20573486940239297 median 0 set size 9148
HindIII-HF 1.339426830429401 median 1 set size 20480
KasI 1.3792725887841568 median 1 set size 16115
NarI 1.3792725887841568 median 1 set size 16115
Nb.BbvCI 0.2725094417468025 median 0 set size 10932
NdeI 0.9950014281633819 median 1 set size 24625
NheI-HF 0.5033482497064331 median 1 set size 20092
NotI-HF 0.5419721349455711 median 1 set size 23919
NsiI-HF 0.7811260274842109 median 1 set size 16119
PacI 

In [233]:
# for the set coverage, we need to know what sequences have ANY coverage by the enzymes we're going to use
training_data = open("train_values.csv")
covered_seqs = []
for index,line in enumerate(training_data):
    sp = line.split(",")
    if index % 10000 == 0:
        print(index)
    sequence = sp[1]
    sequence_name = sp[0]
    if len(sequence) > 2000:
        covered = False
        for enz_name, enz in all_enzymes.items():
            if enz_name in enzymes_to_use:
                count = len(re.findall(enz.regex_seq, line))
                if count > 0:
                    covered = True
        if covered:
            covered_seqs.append(sequence_name)
print(len(covered_seqs))

0
10000
20000
30000
40000
50000
60000
40335


In [240]:
import operator
# greedy set cover adapted from: http://www.martinbroadhurst.com/greedy-set-cover-in-python.html
def set_cover(universe, subsets):
    """Find a family of subsets that covers the universal set"""
    elements = set(e for s in subsets.values() for e in s)
    print(len(elements))
    # Check the subsets cover the universe
    if elements != universe:
        print("No cover")
        return None
    covered = set()
    cover = []
    covering = []
    # Greedily add the subsets with the most uncovered points
    while covered != elements:
        subset = max(subsets.items(), key=lambda s: len(s[1] - covered))
        cover.append(subset[1])
        added_coverage = str(len(subset[1] - covered))
        covering.append(subset[0])
        covered |= subset[1]
        print(str(len(subset[1])) + " " + added_coverage + " " + str(len(covered)))

    return (cover,covering)

universe = set(covered_seqs)
cover = set_cover(universe, enzymes_to_use)
print(cover[1])

40335
26215 26215 26215
24022 8631 34846
26130 3087 37933
20092 1175 39108
17983 540 39648
24625 334 39982
19249 162 40144
18334 85 40229
16115 39 40268
18663 25 40293
16842 14 40307
16119 11 40318
16167 5 40323
23919 5 40328
16954 3 40331
18559 2 40333
13399 1 40334
18944 1 40335
['XhoI', 'EcoRI-HF', 'XbaI', 'BmtI-HF', 'BclI-HF', 'NdeI', 'SacI-HF', 'BsrGI-HF', 'KasI', 'SpeIʤ', 'SacII', 'NsiI-HF', 'AflII', 'NotI-HF', 'RsrII', 'BspDI', 'Bsu36I', 'SphI-HF']


In [239]:
full_set = ""
for enzyme in cover[1]:
    print(all_enzymes[enzyme].sequence.ljust(10) + " " + enzyme.ljust(10) + " " + str(all_enzymes[enzyme].overhang_length()*2))
    full_set += all_enzymes[enzyme].sequence
print(len(full_set))
print(full_set)

CTCGAG     XhoI       -4.0
GAATTC     EcoRI-HF   -4.0
TCTAGA     XbaI       -4.0
GCTAGC     BmtI-HF    4.0
TGATCA     BclI-HF    -4.0
CATATG     NdeI       -2.0
GAGCTC     SacI-HF    4.0
TGTACA     BsrGI-HF   -4.0
GGCGCC     KasI       -4.0
ACTAGT     SpeIʤ      -4.0
CCGCGG     SacII      2.0
ATGCAT     NsiI-HF    4.0
CTTAAG     AflII      -4.0
GCGGCCGC   NotI-HF    -4.0
CGGWCCG    RsrII      -3.0
ATCGAT     BspDI      -2.0
CCTNAGG    Bsu36I     -3.0
GCATGC     SphI-HF    4.0
112
CTCGAGGAATTCTCTAGAGCTAGCTGATCACATATGGAGCTCTGTACAGGCGCCACTAGTCCGCGGATGCATCTTAAGGCGGCCGCCGGWCCGATCGATCCTNAGGGCATGC


In [248]:
import random
bases = ['A','C','G','T']
primer = "".join(random.choices(bases, k=20))
print(primer)

GTCGGATACTACATTCGGAA
